# How to Use Panel for Interactive Display and Layouts

This guide shows you how to use [Panel](https://panel.holoviz.org) to create interactive displays, update plots dynamically, and build sophisticated layouts with hvPlot visualizations.

## Why Use Panel with hvPlot?

Panel extends hvPlot's capabilities by providing:

- **Dynamic updates**: Change plot content without recreating the entire visualization
- **Advanced layouts**: More sophisticated arrangements than simple `+` and `*` operators
- **Interactive widgets**: Add controls like sliders, dropdowns, and buttons
- **Dashboard creation**: Build full applications and dashboards
- **Server deployment**: Easily serve interactive applications

:::{note}
Some of the dynamic functionality in this notebook require a live python process to work.
:::

Let's start with the basics:

In [ ]:
import panel as pn
import hvplot.pandas # noqa

# Enable Panel extension for notebooks
pn.extension('tabulator')

# Load sample data from hvsampledata
earthquakes = hvplot.sampledata.earthquakes("pandas")
penguins = hvplot.sampledata.penguins("pandas").dropna()
stocks = hvplot.sampledata.stocks("pandas")

## Basic Panel Usage

### Creating a Panel Pane

The fundamental concept in Panel is a "pane" - a container that can hold and display objects:

In [ ]:
# Create a simple plot
earthquake_magnitude_plot = earthquakes.hvplot.line(
    x='time',
    y='mag',
    title="Earthquake Magnitude Over Time",
    color='red',
)

# Wrap it in a Panel pane
pane = pn.panel(earthquake_magnitude_plot)
pane

### Dynamic Plot Updates

One of Panel's key features is the ability to update plot content dynamically:

In [ ]:
earthquake_depth_plot = earthquakes.hvplot.line(
    x='time',
    y='depth',
    title="Earthquake Depth Over Time",
)

pane.object = earthquake_depth_plot
print("Pane updated with earthquake depth data - check the plot above!")

### In-Place Updates with Overlays

You can also update the plot content in-place using overlay operations:

In [ ]:
# Add the earthquake magnitude plot as an overlay
pane.object *= earthquake_magnitude_plot
print("Added earthquake magnitude as overlay - now showing both trends!")

## Advanced Layouts with Panel

### Column and Row Layouts

Panel provides more sophisticated layout options than hvPlot's basic `+` operator:

In [ ]:
# Create multiple plots

size_opts = {"width": 350, "height": 250}

plot1 = earthquakes.hvplot(
    x='time', y='mag',
    title="Magnitude", **size_opts,
)

plot2 = earthquakes.hvplot(
    x='time', y='depth',
    title="Depth", **size_opts,
)

plot3 = earthquakes.hvplot.scatter(
    x='lon', y='lat',
    title="Geographic Distribution", **size_opts,
)

plot4 = earthquakes.hvplot.hist(
    'mag', bins=20,
    title="Magnitude Distribution", **size_opts,
)

# Create a row layout
row_layout = pn.Row(plot1, plot2, plot3)
row_layout

In [ ]:
# Create a column layout
column_layout = pn.Column(plot1, plot2, plot3)
column_layout

In [ ]:
# Create a nested layout (2x2 grid)
grid_layout = pn.Column(
    pn.Row(plot1, plot2),
    pn.Row(plot3, plot4)
)
grid_layout

### Tabs for Organized Content

Tabs are perfect for organizing multiple related visualizations:

In [ ]:
# Create different types of plots for the penguin data
size_opts = {"width": 600, "height": 400}

penguin_scatter = penguins.hvplot.scatter(
    x='bill_length_mm', y='bill_depth_mm',
    by='species', title="Bill Dimensions",
    **size_opts,
)

penguin_hist = penguins.hvplot.hist(
    'body_mass_g', by='species',
    title="Body Mass Distribution",
    alpha=0.7, **size_opts,
)

penguin_box = penguins.hvplot.box(
    y='flipper_length_mm', by='species',
    title="Flipper Length by Species",
    **size_opts,
)

# Create tabbed interface
tabs = pn.Tabs(
    ("Scatter Plot", penguin_scatter),
    ("Histogram", penguin_hist),
    ("Box Plot", penguin_box)
)

tabs

## Interactive Widgets and Controls

### Simple Widget Example

Let's create a dropdown to switch between different earthquake metrics:

In [ ]:
# Get available earthquake numeric columns
earthquake_columns = ['mag', 'depth']

# Create a dropdown widget
earthquake_selector = pn.widgets.Select(
    name='Earthquake Metric',
    value=earthquake_columns[0],
    options=earthquake_columns,
    width=200
)

# Function to create plot based on selection
def create_earthquake_plot(metric):
    return earthquakes.hvplot(
        x='time',
        y=metric,
        title=f"{metric.title()} Over Time",
        line_width=3
    )

# Create reactive plot that updates when widget changes
interactive_plot = pn.bind(create_earthquake_plot, earthquake_selector)

# Combine widget and plot
widget_layout = pn.Column(
    "## Interactive Earthquake Data Explorer",
    earthquake_selector,
    interactive_plot
)

widget_layout

### Multiple Widgets Example

Let's create a more complex example with multiple controls for the penguin data:

In [ ]:
# Create multiple widgets
x_axis = pn.widgets.Select(
    name='X Axis',
    value='bill_length_mm',
    options=['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
)

y_axis = pn.widgets.Select(
    name='Y Axis',
    value='bill_depth_mm',
    options=['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
)

plot_type = pn.widgets.Select(
    name='Plot Type',
    value='scatter',
    options=['scatter', 'line', 'bar']
)

color_by = pn.widgets.Select(
    name='Color By',
    value='species',
    options=['species', 'island', 'sex', None]
)

alpha_slider = pn.widgets.FloatSlider(
    name='Transparency',
    start=0.1,
    end=1.0,
    value=0.7,
    step=0.1
)

# Function to create plot based on all widgets
def create_penguin_plot(x, y, plot_kind, color, alpha):
    plot_method = getattr(penguins.hvplot, plot_kind)

    kwargs = {
        'x': x,
        'y': y,
        'title': f"{plot_kind.title()} Plot: {y} vs {x}",
        'width': 600,
        'height': 400,
        'alpha': alpha
    }

    if color:
        kwargs['by'] = color

    return plot_method(**kwargs)

# Bind the function to all widgets
complex_interactive_plot = pn.bind(
    create_penguin_plot,
    x_axis, y_axis,
    plot_type, color_by, alpha_slider
)

# Create control panel
controls = pn.Card(
    x_axis, y_axis, plot_type, color_by, alpha_slider,
    title="Plot Controls",
)

# Combine controls and plot side by side
complex_layout = pn.Row(
    controls,
    pn.Column(
        "## Dynamic Penguin Data Explorer",
        complex_interactive_plot
    )
)

complex_layout

## Dashboard Creation

### Multi-Dataset Dashboard

Let's create a comprehensive dashboard combining multiple datasets:

In [ ]:
# Create summary statistics
earthquake_summary = pn.widgets.Tabulator(
    earthquakes.describe().round(2),
    width=400,
    height=200
)

penguin_summary = pn.widgets.Tabulator(
    penguins.describe().round(2),
    width=400,
    height=200
)

# Create overview plots
earthquake_overview = earthquakes.hvplot.line(
    x='time', y=['mag', 'depth'],
    title="Earthquake Trends Overview",
    width=500, height=300,
)

penguin_overview = penguins.hvplot.scatter(
    x='bill_length_mm', y='body_mass_g',
    by='species', title="Penguin Overview",
    width=500, height=300,
)

summary = pn.pane.Markdown("""
        ## Multi-Dataset Analytics Dashboard

        This dashboard demonstrates:
        - Multiple dataset integration
        - Tables with summary statistics
        - Responsive layout
        - Mixed content types (plots, tables, text)
        """)

tables = pn.Row(
    earthquake_summary,
    penguin_summary
)

plots = pn.Row(
        earthquake_overview,
        penguin_overview
    )

layout = pn.Column(summary, tables, plots)

layout

## Panel Templates and Styling

### Using Built-in Templates

Panel provides several built-in templates for professional-looking applications:

In [ ]:
# Material Design Template
material_template = pn.template.MaterialTemplate(
    title="Material Design Dashboard",
    sidebar=[
        "## Controls",
        earthquake_selector,
        pn.widgets.Button(name="Refresh", button_type='primary')
    ],
    main=[interactive_plot],
    header_background='#3f51b5'
)

material_template.servable();

:::{tip}
You can use `.show()` to display the dashboard in a new tab.
:::

## Summary

Panel extends hvPlot's capabilities in several key ways:

### Key Features:
- **Dynamic Updates**: Change plot content without page reloads
- **Advanced Layouts**: Rows, columns, tabs, and templates
- **Interactive Widgets**: Dropdowns, sliders, buttons, and more
- **Dashboard Creation**: Professional applications with minimal code

### When to Use Panel:
- Building interactive dashboards
- Creating applications for non-technical users
- Need for real-time data updates
- Complex layouts beyond simple plot combinations
- Deploying web applications

### Quick Reference:

| Task | Panel Code |
|------|------------|
| Basic pane | `pn.panel(plot)` |
| Update plot | `pane.object = new_plot` |
| Row layout | `pn.Row(plot1, plot2)` |
| Column layout | `pn.Column(plot1, plot2)` |
| Tabs | `pn.Tabs(("Tab1", plot1), ("Tab2", plot2))` |
| Interactive | `pn.bind(function, widget)` |

:::{seealso}
- Check out other [Panel templates](https://panel.holoviz.org/tutorials/basic/templates.html) here

- Check out the full [Panel documentation](https://panel.holoviz.org) for advanced features!
:::